In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:1800px !important; }</style>"))

import pandas as pd
pd.options.display.max_colwidth = 2999
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.options.display

In [ ]:
from tempfile import NamedTemporaryFile
import subprocess
import pandas as pd

In [ ]:
def disambiguate_word_sense(text, vocab="vocab.100000.txt", model="model.ckpt-100000"):
    with NamedTemporaryFile() as f:        
        f.write(text.encode('utf-8'))
        f.flush()
        cmd = ("python wsd.py --mode topk --vocab_path models/{} --input_path {} --checkpoint_path models/{}".format(vocab, f.name, model))
        proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
        return proc.communicate()[0].decode("utf-8").split('\n')[:-1]

In [ ]:
nick_sentences = [
    (('',), "Business leaders proposed a policy to increase corporate control"),
    (('',), "Leaders in Europe have invested in renewable energy industries")]

general_sentences = [
    (('',), "It was very dark night, there was no light"),
    (('',), "My new phone, iphone 5s, is very light"),             
    (('',),"Fast tanks used during world war II by germans were called Tigers"),
    (('',),"Golf player Tiger Woods had great golf season"),
    (('',),"Tigers are dangerous animals and they live in Asia planes")]

domain_sentences = [
    (('',),"I am unhappy that"),
    (('',),"He is dissatisfied with"),
    (('',),"I should switch back to Verizon"),                           
    (('',),"This android is better than iphone"),
    (('',),"That signal so called booster isn't working"),                          
    (('',),"It's outrage - why's that so expensive")]   

jan_sentences = [
    (('Customer', 'Problem', 'Reception'), "i'm having reception problems"),
    (('Agent', 'Offer', 'iPhone'), 'interest you in the iphone'),
    (('Customer', 'Solution Rebuttal'), "don't think this is going to help"),
    (('Agent', 'Hold'), 'please hold'),
    (('Agent', 'Offer', 'Signal Booster'), 'you need a signal booster'),
    (('Customer', 'Anger'), 'this is outrageous!'),
    (('Customer', 'Churn', 'Competitor'), 'switch back to sprint'),
    (('Agent', 'Empathy'), "i'm sorry that you feel this way,"),
    (('Agent', 'Solutions Exhausted'), 'nothing more could do'),
    (('Customer', 'Objection', 'Price'), "don't want to pay extra to have"),
    (('Agent', 'Empathy'), 'I understand how this can be upsetting'),
    (('Agent', 'Confidence'), 'Let me propose I can'),
    (('Agent', 'Offer', '1 month free'), 'offer you one month free'),
    (('Customer', 'Agreement'), 'that is more like it I can agree to that'),
    (('Customer', 'Gratitude'), 'thank you'),
    (('Agent', 'Request for Evaluation'), 'have i assisted you with all of your issues today?'),
    (('Customer', 'Positive Feedback'), 'thanks very much for your help, you were great'),
    (('Customer', 'Gratitude'), 'thanks')
    ]
    
sentences = nick_sentences + general_sentences + jan_sentences + general_sentences

In [ ]:
from collections import defaultdict
from tqdm import tqdm
presentation_dict = defaultdict(list)
for annotation, sentence in tqdm(sentences):
    text = sentence
    splitted_text = text.split()
    disambiguated_text = disambiguate_word_sense(text)
    if len(splitted_text) == len(disambiguated_text):        
        annotation_text = '_'.join(annotation)
        for word, propositions in zip(splitted_text, disambiguated_text):
            presentation_dict['words'].append(word)            
            presentation_dict['propositions'].append(propositions)        
            presentation_dict['annotation'].append(annotation_text)
            presentation_dict['original_sentence'].append(sentence)

In [ ]:
presentation_df = pd.DataFrame.from_dict(presentation_dict)

In [ ]:
presentation_df

In [ ]:
with open('presentation_of results.html', 'w') as f:
    f.write(presentation_df.to_html())